In [ ]:
""""
Finales Modell wird geschätzt - Parameter auf Grundlage des Hyperparametertrainings
Zusammenfassung:
- Trainingsdatensatz wird geladen
- Ein Tokenizer geschätzt
- Datensatz wird in einem Trainings- und Validerungsdatensatz geteilt
- Oversampling wird durchgeführt
- Y Daten werden faktorisiert
- Für alle Daten wird ein 4-D Tensor erstellt
- Netz wird definiert
- Netz wird trainiert und Daten werden gespeichert
- Mit dem trainerten Netz wird der Testsatensatz geschätzt

env: tf-gpu

Auführen mit:
Jupyter Notebook

Eingabe: Tweets_final_corpus_sentiment_clean_test.json

Ausgabe:
Ergebnis finales  CNN und Test-Accuracy

@author: Paul Drecker

"""

In [ ]:
# Tensorboard laden - je nach Browserversion keine Darstellung im jupyter möglich - öffnen über cmd
%load_ext tensorboard

In [ ]:
#Laden der Pakete
import tensorflow.keras.layers
import datetime
import tensorflow as tf
import keras
import pandas as pd
import os
import numpy as np
import pickle
from sklearn.model_selection import train_test_split
from nltk.tokenize.treebank import TreebankWordDetokenizer
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras import Model, Sequential
from tensorflow.keras.layers import Activation, Dense, Embedding, GlobalAveragePooling1D
from tensorboard.plugins.hparams import api as hp
from imblearn.over_sampling import RandomOverSampler

In [ ]:
# working directory auf den Ordner Masterarbeit festlegen
os.chdir(r'C:\Users\Paul\Documents\Uni\Masterarbeit\Final_Code\Masterarbeit')

In [ ]:
#Daten Laden
Tweets_final_clean = pd.read_json(os.getcwd() + r"\Datengewinnung\Trainingsdaten\Tweets_final_corpus_sentiment_clean_train.json")

In [ ]:
# Tokenizer auf den Trainigsdatensatz anpassen
token = tf.keras.preprocessing.text.Tokenizer(num_words=10000,oov_token='<UNK>' )
token.fit_on_texts(Tweets_final_clean['x_train'])

In [ ]:
# Tokenizer anwenden
Tweets_text = token.texts_to_sequences(Tweets_final_clean['x_train'])

In [ ]:
# Alle Tweets in eine Länge bringen
Tweets_text = tf.keras.preprocessing.sequence.pad_sequences(Tweets_text, maxlen=20, padding='post')

In [ ]:
# In Trainings- und Validerungsdatensatz aufteilen
X_train, X_test, y_train, y_test = train_test_split(Tweets_text,Tweets_final_clean['y_train'], test_size=0.2, random_state=123,stratify=Tweets_final_clean['y_train'], shuffle=True)

In [ ]:
# Oversampling durchführen - zufallsbasiert
oversample = RandomOverSampler(sampling_strategy='auto',random_state=123)
X_train_over, y_train_over = oversample.fit_resample(X_train, y_train)
sample_text = pd.DataFrame(X_train_over, y_train_over).sample(frac=1).reset_index(drop=False)

In [ ]:
# Traindatensätze umbenenen 
y_train_over = sample_text['y_train']
del(sample_text['y_train'])
X_train_over = sample_text.to_numpy()

In [ ]:
# Labels Faktorisieren
y_train = pd.factorize(y_train_over)
y_train =  tf.constant(y_train[0] , dtype = tf.float32)
y_train =  tf.expand_dims(y_train, 1)

# Labels Faktorisieren
y_test = pd.factorize(y_test)
y_test =  tf.constant(y_test[0] , dtype = tf.float32)
y_test =  tf.expand_dims(y_test, 1)

# In Kategorievariablen umwandeln
y_test = tf.keras.utils.to_categorical(y_test)
y_train = tf.keras.utils.to_categorical(y_train)


# Für Trainingsdaten 3-D Tensor erstellen
X_train_3dtensor = tf.stack(list(X_train_over), axis=0)
X_train_3dtensor.get_shape()
X_train = None
del(X_train)
# Für Trainingsdaten 4-D Tensor erstellen
X_train_4dtensor = tf.expand_dims(X_train_3dtensor, -1) 
X_train_4dtensor.get_shape()

# Für Validerungssdaten 3-D Tensor erstellen
X_test_3dtensor = tf.stack(list(X_test), axis=0)
X_test = None
del(X_test)
# Für Validerungssdaten 4-D Tensor erstellen
X_test_4dtensor = tf.expand_dims(X_test_3dtensor, -1) 




In [ ]:
# Gespeicherte Modelle in der Session löschen - vermeiden von doppelten Strukturen
tf.keras.backend.clear_session()

In [ ]:
# Modell wird definiert
# Filtergößen festlegen
filter_sizes=[3,4,5]

# Embedding Schicht
input_layer = tf.keras.layers.Input(shape=(20,1,), name="input_layer")
emb_layer = tf.keras.layers.Embedding(10000, 300)(input_layer)
emb_trans = tf.transpose(emb_layer,[0,1,3,2])
emb = tf.squeeze(emb_trans, 3)
emb = tf.transpose(emb,[0,2,1])

# Convolution Schicht
con_3 = tf.keras.layers.Conv1D(120, (filter_sizes[0]))(emb)
relu_3 =tf.keras.layers.ReLU()(con_3 )

con_4 = tf.keras.layers.Conv1D(120, (filter_sizes[1]))(emb)
relu_4 =tf.keras.layers.ReLU()(con_4 )

con_5 = tf.keras.layers.Conv1D(120, (filter_sizes[2]))(emb)
relu_5 =tf.keras.layers.ReLU()(con_5 )

# Maxpooling
max_3 = tf.keras.layers.MaxPool1D(pool_size=(con_3.get_shape()[1]) ,strides=(1), padding='valid')(relu_3)
max_4 = tf.keras.layers.MaxPool1D(pool_size=(con_4.get_shape()[1] ),strides=(1), padding='valid')(relu_4)
max_5 = tf.keras.layers.MaxPool1D(pool_size=(con_5.get_shape()[1]),strides=(1), padding='valid')(relu_5)

# Flatting
merged = tf.keras.layers.Concatenate(axis=1)([max_3,max_4, max_5])
flatten = tf.keras.layers.Flatten()(merged)

# Dropout Schicht
dropout = tf.keras.layers.Dropout(0.5)(flatten)

# Dense Schicht und Softmatrix
dense_layer = tf.keras.layers.Dense(units=2, name="dense_layer", kernel_regularizer=tf.keras.regularizers.l2(0.0001))(dropout)
output_layer = tf.keras.layers.Softmax(name="output_layer")(dense_layer)
model = tf.keras.models.Model(input_layer, output_layer, name="model")
model.compile(loss=tf.keras.losses.BinaryCrossentropy(),
              optimizer=tf.keras.optimizers.Adam(0.0001),
              metrics=[tf.keras.metrics.BinaryAccuracy()])


model.summary()
    
    

In [ ]:
# Speicher festlegen
logdir = os.getcwd() +'\Sentiment_model\CNN-final/logs/'
session_num = 0

logs = os.getcwd() +'\Sentiment_model\CNN-final/logs/fit/' 
#Training
model.fit(X_train_4dtensor, y_train, epochs=6,shuffle=True,verbose=1,validation_data=(X_test_4dtensor, y_test)  ,batch_size= 32 ,callbacks=[
        
        tf.keras.callbacks.TensorBoard(
        log_dir=logs, histogram_freq=1, write_graph=True,
        write_images=False, update_freq='epoch', profile_batch=2,
        embeddings_freq=1, embeddings_metadata=None)
           ]) 
    




In [ ]:
# Testdatensatz laden
Final_test = pd.read_json(os.getcwd() + r"\Datengewinnung\Trainingsdaten\Tweets_final_corpus_sentiment_clean_test.json")
# Tokenizer anwenden - Dieser wurde auf Basis der Trainingsdaten bestimmt
Final_test_x = token.texts_to_sequences(Final_test['x_test'])
# Alle Tweets in eine Länge bringen
Final_test_x = tf.keras.preprocessing.sequence.pad_sequences(Final_test_x, maxlen=20, padding='post')
# Labels Faktorisieren
y_test_final = pd.factorize(Final_test['y_test'])


In [ ]:
y_test_final  =  tf.constant(y_test_final[0] , dtype = tf.float32)
y_test_final  =  tf.expand_dims(y_test_final, 1)
y_test_final  = tf.keras.utils.to_categorical(y_test_final)
 
# Für Testdaten 3-D Tensor erstellen
X_test_3dtensor_final = tf.stack(list(Final_test_x), axis=0)
X_test = None
del(X_test)
# Für Testdaten 4-D Tensor erstellen
X_test_4dtensor_final = tf.expand_dims(X_test_3dtensor_final, -1) 


In [ ]:
#Modell auf Testdaten anwenden
model.evaluate(X_test_4dtensor_final, y_test_final)